# PROJET 9 : Produire une étude de marché avec python

## Introduction

### Contexte:
*'La poule qui chante'* est une entreprise francaise d'agroalimentaire fictive qui souhaite se développer à l'international.</br> 
Dans ce contexte, ils souhaitent avoir une première analyse des pays qui peuvent etre interessant pour l'export de poulet.

### Mission: 
- Effectuer une première analyse des groupements par pays pour une exportation de poulet 
- Approndir l'étude de marché

### Plan:
Le projet est séparé en deux notebooks jupyter. 
</br>
1. Dans ce notebook nous faisons l'exploration des jeux de données et la preparation des données. 
</br>
2. Dans le fichier suivant,
on utilisera des methodes d'apprentissage non supervisées (**CAH** , **Kmeans**) et la **PCA** ( Principal Component Analysis, ou PCA en francais) pour regroupé les pays selons les features selectionnées, et analyser les groupes.

## Préparation des données 
Cette partie inclus la récupération des données dans les jeux de données, le nettoyage des données.</br> 
A la fin de la préparation, on aura un fichier qui sera utilisé pour l'analyse.

### Quelles données utiliser ?

Afin d'avoir une analyse interessante, on veux des données qui ont un sens dans le contexte de l'exportation de poulet.
Les informations qui peuvent nous interesser sont:

1. Le régime alimentaire 
    + la disponibilité alimentaire par habitant ( variable "Disponibilité alimentaire (Kcal/personne/jour)")
    + La proportion de protéines d'origine de volaille: Calculer avec la formule: *protéine de volaille /toute les protéines*

2. Le TDI (Taux de dépendance aux importation) = *importation / disponibilité intérieur x 100*
3. Le TAS (Taux d'auto-suffisance) = *Production / Disponibilité intérieur x 100*

4. La production de volaille:
    - Quantité production par habitant: *Production volaille / population en 2017*
    - Production: Quantité de la production de volaille

5. Membre de l'EEE (L'Espace économique européen (EEE))
    - Cela est interessant car libre circulation des marchandises, donc non soumis à des taxes d'importation.

6. L'indice de risque : Ce sera la variable 'CountryRisk' lors de l'analyse.
    - Cela permet de noter un pays selon le risque d'investissement.
    - Une note haute indique un pays fiable.

7. L´évolution sur 5 ans (2012-2017): 
   - Taux de croissance de la population sur 5 ans 
   - Taux de croissance du pouvoir d'achat (PIB/ HAB PPA)
8. La géographie
     - Distance à la France
         - La proximité d'un pays permet de minimiser les coût de transports.

### Importation des bibliotheques utilisées

In [652]:
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt

### Preparation des données de population
Dans le jeux de données "Population_2000_2018.csv", on veut recuperer la population par pays en 2017, et calculer la croissance de population sur 5 ans

In [653]:
# Importer le fichier population 
population = pd.read_csv('Population_2000_2018.csv')
population.head(2)

,Code Domaine,Domaine,Code zone,Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole,Note
0,OA,Séries temporelles annuelles,2,Afghanistan,511,Population totale,3010,Population-Estimations,2000,2000,1000 personnes,20779.953,X,Sources internationales sûres,NaN
1,OA,Séries temporelles annuelles,2,Afghanistan,511,Population totale,3010,Population-Estimations,2001,2001,1000 personnes,21606.988,X,Sources internationales sûres,NaN


In [654]:
#Enlever les colonnes dont on n'a pas besoin
population_1= population.copy()
population_1.drop(columns=['Domaine','Code Domaine','Code zone','Code Élément','Élément','Code Produit','Produit','Code année','Symbole','Description du Symbole','Note','Unité'], axis=1, inplace = True)
population_1.head(2)

,Zone,Année,Valeur
0,Afghanistan,2000,20779.953
1,Afghanistan,2001,21606.988


#### Ajout d'une nouvelle variable pour notre analyse : "population_2017"

L'unité pour cette variable est le millier d'habitant. (1000 habitants)

In [655]:
# 這個檔案為了後續merged用
population_2017 = population_1.copy()
mask = population_2017["Année"] == 2017 
population_2017[mask]
population_2017 = population_2017[mask]
population_2017 = population_2017.loc[:,['Zone','Valeur']]
population_2017.rename(columns={'Valeur':'population_2017'}, inplace = True)
population_2017.head(2)

,Zone,population_2017
17,Afghanistan,36296.113
36,Afrique du Sud,57009.756


In [656]:
population_2017.isnull().sum()

Zone               0
population_2017    0
dtype: int64

#### La croissance démographique en % entre 2012 et 2017
Ici, on calcule la croissance démographique. 

In [657]:
# Calculer une nouvelle variable pour notre analyse : la croissance démographique en % entre 2012 et 2017
population_2 = population_1.copy()
population_2 =  population_2.pivot(index = 'Zone', columns = 'Année', values = 'Valeur')
population_2.head()


Année,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
Zone,,,,,,,,,,,,,,,,,,,
Afghanistan,20779.953,21606.988,22600.770,23680.871,24726.684,25654.277,26433.049,27100.536,27722.276,28394.813,29185.507,30117.413,31161.376,32269.589,33370.794,34413.603,35383.032,36296.113,37171.921
Afrique du Sud,44967.708,45571.274,46150.913,46719.196,47291.610,47880.601,48489.459,49119.759,49779.471,50477.011,51216.964,52003.755,52832.658,53687.121,54544.186,55386.367,56207.646,57009.756,57792.518
Albanie,3129.243,3129.704,3126.187,3118.023,3104.892,3086.810,3063.021,3033.998,3002.678,2973.048,2948.023,2928.592,2914.096,2903.790,2896.305,2890.513,2886.438,2884.169,2882.740
Algérie,31042.235,31451.514,31855.109,32264.157,32692.163,33149.724,33641.002,34166.972,34730.608,35333.881,35977.455,36661.445,37383.895,38140.133,38923.692,39728.025,40551.392,41389.189,42228.408
Allemagne,81400.882,81453.885,81535.122,81614.380,81646.474,81602.741,81472.226,81277.830,81065.752,80899.961,80827.002,80855.632,80972.628,81174.367,81450.378,81787.411,82193.768,82658.409,83124.418


In [658]:
# croissance demongraphique sur 5 ans 
population_2['Croissance_popu(%)']= (population_2[2017]-population_2[2012])/(population_2[2012])*100
#Garder seulement croissance_popu
population_2= population_2.loc[:,['Croissance_popu(%)']]
population_2.head()

Année,Croissance_popu(%)
Zone,
Afghanistan,16.477889
Afrique du Sud,7.906280
Albanie,-1.026974
Algérie,10.713956
Allemagne,2.081915


In [659]:
population_2.isnull().sum()

Année
Croissance_popu(%)    2
dtype: int64

In [660]:
# 2 valeurs nulles que nous traiterons au moment des jointures
population_2[population_2.isna().any(axis=1)]

Année,Croissance_popu(%)
Zone,
Serbie-et-Monténégro,NaN
Soudan (ex),NaN


#### Jointure des informations sur la population

In [661]:
df_popu = population_2.merge(population_2017,how='inner',on='Zone')
df_popu.head()

,Zone,Croissance_popu(%),population_2017
0,Afghanistan,16.477889,36296.113
1,Afrique du Sud,7.906280,57009.756
2,Albanie,-1.026974,2884.169
3,Algérie,10.713956,41389.189
4,Allemagne,2.081915,82658.409


In [48]:
df_popu.isnull().sum()

Zone                  0
Croissance_popu(%)    0
population_2017       0
dtype: int64

### Traitement des données de "disponibilité alimentaire"
On lit le jeux de données *"DisponibiliteAlimentaire_2017.csv"*

On va calculer les variable suivantes:
1. **TDI** (Taux de dépendance aux importation) = *importation / disponibilite interieur * 100*
2. **TAS** (Taux d'auto-suffisance) = *production /disponibilite interieux * 100*

 (disponibilité intérieur = Production + importation - Exportation + variation de stock)

In [662]:
# Importer le fichier disponibilité_alimentaire_2017 au format csv
disponibilité_alim_2017 = pd.read_csv('DisponibiliteAlimentaire_2017.csv')
disponibilité_alim_2017.head()

,Code Domaine,Domaine,Code zone,Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole
0,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5511,Production,2511,Blé et produits,2017,2017,Milliers de tonnes,4281.0,S,Données standardisées
1,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5611,Importations - Quantité,2511,Blé et produits,2017,2017,Milliers de tonnes,2302.0,S,Données standardisées
2,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5072,Variation de stock,2511,Blé et produits,2017,2017,Milliers de tonnes,-119.0,S,Données standardisées
3,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5911,Exportations - Quantité,2511,Blé et produits,2017,2017,Milliers de tonnes,0.0,S,Données standardisées
4,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5301,Disponibilité intérieure,2511,Blé et produits,2017,2017,Milliers de tonnes,6701.0,S,Données standardisées


In [663]:
disponibilité_alim_2017['Élément'].unique()

array(['Production', 'Importations - Quantité', 'Variation de stock',
       'Exportations - Quantité', 'Disponibilité intérieure',
       'Aliments pour animaux', 'Semences', 'Pertes', 'Résidus',
       'Nourriture',
       'Disponibilité alimentaire en quantité (kg/personne/an)',
       'Disponibilité alimentaire (Kcal/personne/jour)',
       'Disponibilité de protéines en quantité (g/personne/jour)',
       'Disponibilité de matière grasse en quantité (g/personne/jour)',
       'Traitement', 'Autres utilisations (non alimentaire)',
       'Alimentation pour touristes'], dtype=object)

In [664]:
disponibilité_alim_2017['Produit'].unique()

array(['Blé et produits', 'Riz et produits', 'Orge et produits',
       'Maïs et produits', 'Seigle et produits', 'Avoine',
       'Millet et produits', 'Sorgho et produits', 'Céréales, Autres',
       'Pommes de Terre et produits', 'Ignames', 'Racines nda',
       'Sucre, canne', 'Sucre, betterave', 'Sucre Eq Brut',
       'Edulcorants Autres', 'Miel', 'Haricots', 'Pois',
       'Légumineuses Autres et produits', 'Noix et produits', 'Soja',
       'Arachides Decortiquees', 'Graines de tournesol',
       'Graines Colza/Moutarde', 'Graines de coton', 'Coco (Incl Coprah)',
       'Sésame', 'Olives', 'Plantes Oleiferes, Autre', 'Huile de Soja',
       "Huile d'Arachide", 'Huile de Tournesol',
       'Huile de Colza&Moutarde', 'Huile Graines de Coton',
       'Huile de Palmistes', 'Huile de Palme', 'Huile de Coco',
       'Huile de Sésame', "Huile d'Olive", 'Huile de Son de Riz',
       'Huile de Germe de Maïs', 'Huil Plantes Oleif Autr',
       'Tomates et produits', 'Oignons', 'Légumes, 

In [665]:
disponibilité_alim_2017['Unité'].unique()

array(['Milliers de tonnes', 'kg', 'Kcal/personne/jour',
       'g/personne/jour'], dtype=object)

Il y a 17 genres de données dans la colonne d'Elément. 
On sélectionne la viande de volailles dans la colonne de 'Produit' pour notre analyse.

In [666]:
# Afficher uniquement les colonnes dont nous avons besoin
dispo_2017=disponibilité_alim_2017.loc[:,['Zone','Élément','Produit','Valeur','Unité']]
dispo_2017.head()

,Zone,Élément,Produit,Valeur,Unité
0,Afghanistan,Production,Blé et produits,4281.0,Milliers de tonnes
1,Afghanistan,Importations - Quantité,Blé et produits,2302.0,Milliers de tonnes
2,Afghanistan,Variation de stock,Blé et produits,-119.0,Milliers de tonnes
3,Afghanistan,Exportations - Quantité,Blé et produits,0.0,Milliers de tonnes
4,Afghanistan,Disponibilité intérieure,Blé et produits,6701.0,Milliers de tonnes


#### Disponibilite alimentaire


In [667]:
# Conserver uniquement les données concernant l'element de 'Disponibilité de protéines en quantité'
dispo_alimentaire = dispo_2017.loc[dispo_2017['Élément'] == 'Disponibilité alimentaire (Kcal/personne/jour)'].copy()
dispo_alimentaire.head()

,Zone,Élément,Produit,Valeur,Unité
11,Afghanistan,Disponibilité alimentaire (Kcal/personne/jour),Blé et produits,1331.0,Kcal/personne/jour
24,Afghanistan,Disponibilité alimentaire (Kcal/personne/jour),Riz et produits,135.0,Kcal/personne/jour
38,Afghanistan,Disponibilité alimentaire (Kcal/personne/jour),Orge et produits,5.0,Kcal/personne/jour
52,Afghanistan,Disponibilité alimentaire (Kcal/personne/jour),Maïs et produits,11.0,Kcal/personne/jour
77,Afghanistan,Disponibilité alimentaire (Kcal/personne/jour),Millet et produits,2.0,Kcal/personne/jour


In [668]:
dispo_alimentaire = dispo_alimentaire.groupby(['Zone']).sum('Valeur')
dispo_alimentaire = dispo_alimentaire.reset_index()
dispo_alimentaire.head()

,Zone,Valeur
0,Afghanistan,1997.0
1,Afrique du Sud,2987.0
2,Albanie,3400.0
3,Algérie,3345.0
4,Allemagne,3559.0


In [669]:
dispo_alimentaire.rename(columns={'Valeur':'Disponibilité alimentaire (Kcal/personne/jour)'}, inplace = True)

In [670]:
dispo_alimentaire.head()

,Zone,Disponibilité alimentaire (Kcal/personne/jour)
0,Afghanistan,1997.0
1,Afrique du Sud,2987.0
2,Albanie,3400.0
3,Algérie,3345.0
4,Allemagne,3559.0


#### Calcul de la proportion de proteines de volaille qui compose la disponibilité alimentaire.
 Proteine de volaille / Disponibilité de protéines en quantité 

In [671]:
# Conserver uniquement les données concernant l'element de 'Disponibilité de protéines en quantité'
dispo_proteine = dispo_2017.loc[dispo_2017['Élément'] == 'Disponibilité de protéines en quantité (g/personne/jour)'].copy()
dispo_proteine.head()

,Zone,Élément,Produit,Valeur,Unité
12,Afghanistan,Disponibilité de protéines en quantité (g/pers...,Blé et produits,35.88,g/personne/jour
25,Afghanistan,Disponibilité de protéines en quantité (g/pers...,Riz et produits,2.58,g/personne/jour
39,Afghanistan,Disponibilité de protéines en quantité (g/pers...,Orge et produits,0.14,g/personne/jour
53,Afghanistan,Disponibilité de protéines en quantité (g/pers...,Maïs et produits,0.27,g/personne/jour
78,Afghanistan,Disponibilité de protéines en quantité (g/pers...,Millet et produits,0.05,g/personne/jour


In [672]:
# Additionner toutes les proteines 加總每日每人可供應到的總蛋白質量(包含動植物)
dispo_proteine1 = dispo_proteine.copy()
dispo_proteine1 = dispo_proteine.groupby(['Zone']).sum('Valeur')
dispo_proteine1.rename(columns={'Valeur':'proteine_tout(g)'}, inplace = True)
dispo_proteine1= dispo_proteine1.reset_index()
dispo_proteine1.head(2)

,Zone,proteine_tout(g)
0,Afghanistan,54.09
1,Afrique du Sud,83.36


In [673]:
# Conserver uniquement les données concernant le produit 'Viande de Volailles' 
dispo_viande_volaille = dispo_2017.loc[dispo_2017['Produit'] == 'Viande de Volailles'].copy()
# Conserver uniquement les données concernant le proteine de 'Viande de Volailles'提取只有禽肉蛋白質量
dispo_proteine_volaille = dispo_proteine.loc[dispo_proteine['Produit'] == 'Viande de Volailles'].copy()
dispo_proteine_volaille.head(2)

,Zone,Élément,Produit,Valeur,Unité
660,Afghanistan,Disponibilité de protéines en quantité (g/pers...,Viande de Volailles,0.54,g/personne/jour
1715,Afrique du Sud,Disponibilité de protéines en quantité (g/pers...,Viande de Volailles,14.11,g/personne/jour


#### Jointure

In [674]:
# Jointure pour le calcul de la proportion de proteines de volaille
dispo_proteine_merged = dispo_proteine_volaille.merge(dispo_proteine1,how='inner', on='Zone' )
dispo_proteine_merged['proteine_volaille(%)']= (dispo_proteine_merged['Valeur'])/(dispo_proteine_merged['proteine_tout(g)'])*100
dispo_proteine1= dispo_proteine_merged.loc[:,['Zone','proteine_volaille(%)']]
dispo_proteine1.head()

,Zone,proteine_volaille(%)
0,Afghanistan,0.998336
1,Afrique du Sud,16.926583
2,Albanie,5.238494
3,Algérie,2.121702
4,Allemagne,7.648698


In [675]:
dispo_proteine1.isnull().sum()

Zone                    0
proteine_volaille(%)    0
dtype: int64

In [676]:
# 把只有禽肉的element做pivot,提取裡面的值,並用fillna把valeur nulles都放上mean值
dispo = dispo_viande_volaille.pivot_table(index = 'Zone', columns = 'Élément', values = 'Valeur').fillna(dispo.mean())
dispo = dispo.reset_index()
dispo.head()

C:\Users\yutin\AppData\Local\Temp\ipykernel_11200\3456985621.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  dispo = dispo_viande_volaille.pivot_table(index = 'Zone', columns = 'Élément', values = 'Valeur').fillna(dispo.mean())


Élément,Zone,Alimentation pour touristes,Aliments pour animaux,Autres utilisations (non alimentaire),Disponibilité alimentaire (Kcal/personne/jour),Disponibilité alimentaire en quantité (kg/personne/an),Disponibilité de matière grasse en quantité (g/personne/jour),Disponibilité de protéines en quantité (g/personne/jour),Disponibilité intérieure,Exportations - Quantité,Importations - Quantité,Nourriture,Pertes,Production,Résidus,Semences,Traitement,Variation de stock
0,Afghanistan,0.217949,0.0,44.117647,5.0,1.53,0.33,0.54,57.0,132.185185,29.0,55.0,2.00000,28.0,0.0,0.0,26.282609,0.0
1,Afrique du Sud,0.000000,0.0,44.117647,143.0,35.69,9.25,14.11,2118.0,63.000000,514.0,2035.0,83.00000,1667.0,0.0,0.0,26.282609,0.0
2,Albanie,0.217949,0.0,44.117647,85.0,16.36,6.45,6.26,47.0,0.000000,38.0,47.0,43.58209,13.0,0.0,0.0,26.282609,4.0
3,Algérie,0.000000,0.0,44.117647,22.0,6.38,1.50,1.97,277.0,0.000000,2.0,264.0,13.00000,275.0,0.0,0.0,26.282609,0.0
4,Allemagne,0.217949,0.0,44.117647,71.0,19.47,4.16,7.96,1739.0,646.000000,842.0,1609.0,43.58209,1514.0,-38.0,0.0,167.000000,-29.0


In [677]:
dispo.isnull().sum()

Élément
Zone                                                             0
Alimentation pour touristes                                      0
Aliments pour animaux                                            0
Autres utilisations (non alimentaire)                            0
Disponibilité alimentaire (Kcal/personne/jour)                   0
Disponibilité alimentaire en quantité (kg/personne/an)           0
Disponibilité de matière grasse en quantité (g/personne/jour)    0
Disponibilité de protéines en quantité (g/personne/jour)         0
Disponibilité intérieure                                         0
Exportations - Quantité                                          0
Importations - Quantité                                          0
Nourriture                                                       0
Pertes                                                           0
Production                                                       0
Résidus                                               

Maintenant, on ajoute les variables TDI et TAS

1. TDI (Taux de dépendance aux importation) 進口率 = importation 除以 disponibilite interieur * 100
2. TAS (Taux d'auto-suffisance) 自給自足率  = production 除以 disponibilite interieux * 100 

In [678]:
dispo['TDI(%)'] = (dispo['Importations - Quantité'])/(dispo['Disponibilité intérieure'])*100
dispo['TAS(%)'] = (dispo['Production'])/(dispo['Disponibilité intérieure'])*100
dispo.head(2)

Élément,Zone,Alimentation pour touristes,Aliments pour animaux,Autres utilisations (non alimentaire),Disponibilité alimentaire (Kcal/personne/jour),Disponibilité alimentaire en quantité (kg/personne/an),Disponibilité de matière grasse en quantité (g/personne/jour),Disponibilité de protéines en quantité (g/personne/jour),Disponibilité intérieure,Exportations - Quantité,Importations - Quantité,Nourriture,Pertes,Production,Résidus,Semences,Traitement,Variation de stock,TDI(%),TAS(%)
0,Afghanistan,0.217949,0.0,44.117647,5.0,1.53,0.33,0.54,57.0,132.185185,29.0,55.0,2.0,28.0,0.0,0.0,26.282609,0.0,50.877193,49.122807
1,Afrique du Sud,0.000000,0.0,44.117647,143.0,35.69,9.25,14.11,2118.0,63.000000,514.0,2035.0,83.0,1667.0,0.0,0.0,26.282609,0.0,24.268178,78.706327


In [679]:
dispo1 = dispo.loc[:,['Zone','Production','TDI(%)','TAS(%)']]
dispo1.head()

Élément,Zone,Production,TDI(%),TAS(%)
0,Afghanistan,28.0,50.877193,49.122807
1,Afrique du Sud,1667.0,24.268178,78.706327
2,Albanie,13.0,80.851064,27.659574
3,Algérie,275.0,0.722022,99.277978
4,Allemagne,1514.0,48.418631,87.061530


#### Jointure
Dans un premier temps , on refait notre dataframe *df_dispo* en fusionnant les informations de proteine1

In [680]:
df_dispo = dispo_proteine1.merge(dispo1,how='inner',on='Zone')
df_dispo.head()

,Zone,proteine_volaille(%),Production,TDI(%),TAS(%)
0,Afghanistan,0.998336,28.0,50.877193,49.122807
1,Afrique du Sud,16.926583,1667.0,24.268178,78.706327
2,Albanie,5.238494,13.0,80.851064,27.659574
3,Algérie,2.121702,275.0,0.722022,99.277978
4,Allemagne,7.648698,1514.0,48.418631,87.061530


Ensuite, on fusionne aussi avec *df_popu*

In [682]:
df_merged1 = df_popu.merge(df_dispo,how='inner',on='Zone')
df_merged1.head()

,Zone,Croissance_popu(%),population_2017,proteine_volaille(%),Production,TDI(%),TAS(%)
0,Afghanistan,16.477889,36296.113,0.998336,28.0,50.877193,49.122807
1,Afrique du Sud,7.906280,57009.756,16.926583,1667.0,24.268178,78.706327
2,Albanie,-1.026974,2884.169,5.238494,13.0,80.851064,27.659574
3,Algérie,10.713956,41389.189,2.121702,275.0,0.722022,99.277978
4,Allemagne,2.081915,82658.409,7.648698,1514.0,48.418631,87.061530


*dispo_alimentaire* contient la dispo. alimentaire par habitant global ( pas seulement volaille). On ajoute cette information dans notre dataframe.

In [683]:
df_merged1 = df_merged1.merge(dispo_alimentaire,how='inner',on='Zone')
df_merged1.head()

,Zone,Croissance_popu(%),population_2017,proteine_volaille(%),Production,TDI(%),TAS(%),Disponibilité alimentaire (Kcal/personne/jour)
0,Afghanistan,16.477889,36296.113,0.998336,28.0,50.877193,49.122807,1997.0
1,Afrique du Sud,7.906280,57009.756,16.926583,1667.0,24.268178,78.706327,2987.0
2,Albanie,-1.026974,2884.169,5.238494,13.0,80.851064,27.659574,3400.0
3,Algérie,10.713956,41389.189,2.121702,275.0,0.722022,99.277978,3345.0
4,Allemagne,2.081915,82658.409,7.648698,1514.0,48.418631,87.061530,3559.0


In [684]:
df_merged1.isnull().sum()

Zone                                              0
Croissance_popu(%)                                0
population_2017                                   0
proteine_volaille(%)                              0
Production                                        0
TDI(%)                                            0
TAS(%)                                            0
Disponibilité alimentaire (Kcal/personne/jour)    0
dtype: int64

df_merged1 : 172 pays 

On enregistre notre dataframe au format csv.

In [685]:
df_merged1.to_csv('dfmerged1.csv', index=False, encoding='utf-8')

### Calcul de la Quantité de Production de Volaille par habitant

Production volaille par habitant = Production de volaille / Population en 2017 
L'unité pour cette variable est le kilogramme. 

In [686]:
# Quantité de volaille consommée issue 
# On multiplie par 1 000 000 pour convertir Milliers de tonnes en kilogramme.
# population_2017 est en milliers d'habitants
df_merged1['Qty_prod_hab'] = (df_merged1['Production']*1000000)/(df_merged1['population_2017']*1000)
df_merged1.head()

,Zone,Croissance_popu(%),population_2017,proteine_volaille(%),Production,TDI(%),TAS(%),Disponibilité alimentaire (Kcal/personne/jour),Qty_prod_hab
0,Afghanistan,16.477889,36296.113,0.998336,28.0,50.877193,49.122807,1997.0,0.771432
1,Afrique du Sud,7.906280,57009.756,16.926583,1667.0,24.268178,78.706327,2987.0,29.240609
2,Albanie,-1.026974,2884.169,5.238494,13.0,80.851064,27.659574,3400.0,4.507364
3,Algérie,10.713956,41389.189,2.121702,275.0,0.722022,99.277978,3345.0,6.644247
4,Allemagne,2.081915,82658.409,7.648698,1514.0,48.418631,87.061530,3559.0,18.316346


### Pouvoir d'achat (PIB / HAB PPA)
Le calcul de l'évolution du PIB sur 5 ans
Le jeux de données utilisé est téléchargeable sur le site de la FAO.
</br>
*Source: FAO*

In [688]:
# On lit les données de 'PIB'
pib = pd.read_csv('PIB_2012_2017.csv')
pib.head(3)

,Code Domaine,Domaine,Code zone (M49),Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole,Note
0,MK,Indicateurs macro,4,Afghanistan,61290,Croissance annuelle US$ par habitant,22008,Produit Intérieur Brut,2012,2012,%,6.597411,E,Valeur estimée,NaN
1,MK,Indicateurs macro,4,Afghanistan,61290,Croissance annuelle US$ par habitant,22008,Produit Intérieur Brut,2013,2013,%,-0.985318,E,Valeur estimée,NaN
2,MK,Indicateurs macro,4,Afghanistan,61290,Croissance annuelle US$ par habitant,22008,Produit Intérieur Brut,2014,2014,%,-3.650301,E,Valeur estimée,NaN


In [689]:
# Il y a 212 lignes dans 'df_pib'  
pib['PIB_5ans(%)'] = pib.groupby(['Zone'])['Valeur'].transform('mean')
df_pib = pib.loc[:,['Zone','PIB_5ans(%)']]
df_pib = df_pib.drop_duplicates('Zone')
df_pib.head()


,Zone,PIB_5ans(%)
0,Afghanistan,-1.556597
6,Afrique du Sud,-3.981301
12,Albanie,0.723746
18,Algérie,-4.103502
24,Allemagne,-0.328481


In [690]:
df_pib.isnull().sum()

Zone           0
PIB_5ans(%)    0
dtype: int64

In [565]:
df_pib.to_csv('pib.csv', index=False, encoding='utf-8')

### Jointure: df_pib + df_merged1


In [691]:
# left join pour garder Taiwan, il y a 172 zones après la jointure.
df_merged2 = df_merged1.merge(df_pib,how='left',on='Zone')
df_merged2.head()

,Zone,Croissance_popu(%),population_2017,proteine_volaille(%),Production,TDI(%),TAS(%),Disponibilité alimentaire (Kcal/personne/jour),Qty_prod_hab,PIB_5ans(%)
0,Afghanistan,16.477889,36296.113,0.998336,28.0,50.877193,49.122807,1997.0,0.771432,-1.556597
1,Afrique du Sud,7.906280,57009.756,16.926583,1667.0,24.268178,78.706327,2987.0,29.240609,-3.981301
2,Albanie,-1.026974,2884.169,5.238494,13.0,80.851064,27.659574,3400.0,4.507364,0.723746
3,Algérie,10.713956,41389.189,2.121702,275.0,0.722022,99.277978,3345.0,6.644247,-4.103502
4,Allemagne,2.081915,82658.409,7.648698,1514.0,48.418631,87.061530,3559.0,18.316346,-0.328481


In [692]:
# On trouve qu'il y a une valeur nulle pour Taiwan
df_merged2[df_merged2.isna().any(axis=1)]

,Zone,Croissance_popu(%),population_2017,proteine_volaille(%),Production,TDI(%),TAS(%),Disponibilité alimentaire (Kcal/personne/jour),Qty_prod_hab,PIB_5ans(%)
33,"Chine, Taiwan Province de",1.403399,23674.546,12.845642,652.0,20.509554,83.057325,2976.0,27.540127,NaN


In [693]:
# Fillna la moyenne pour la valeur nulles dans la colonne de 'PIB_5ans(%)'
df_merged2["PIB_5ans(%)"] = df_merged2["PIB_5ans(%)"].fillna(df_merged2["PIB_5ans(%)"].mean())
df_merged2.head(2)

,Zone,Croissance_popu(%),population_2017,proteine_volaille(%),Production,TDI(%),TAS(%),Disponibilité alimentaire (Kcal/personne/jour),Qty_prod_hab,PIB_5ans(%)
0,Afghanistan,16.477889,36296.113,0.998336,28.0,50.877193,49.122807,1997.0,0.771432,-1.556597
1,Afrique du Sud,7.906280,57009.756,16.926583,1667.0,24.268178,78.706327,2987.0,29.240609,-3.981301


In [694]:
df_merged2.isnull().sum()

Zone                                              0
Croissance_popu(%)                                0
population_2017                                   0
proteine_volaille(%)                              0
Production                                        0
TDI(%)                                            0
TAS(%)                                            0
Disponibilité alimentaire (Kcal/personne/jour)    0
Qty_prod_hab                                      0
PIB_5ans(%)                                       0
dtype: int64

# L'indice de risque (Country Risk)
On utilise les données réalisées par l'entreprise privée *Coface* qui sont en accès libre.
L'indice de risque évalue 160 pays, calculé à partir de données macroéconomiques, financières et politiques. 
C’est un outil interessant pour les entreprises qui souhaitent se tourner vers l'international.
</br>
*Source : coface.fr*

In [570]:
# 爬蟲抓取資料
df_country_risk = pd.read_html('https://www.coface.fr/Etudes-economiques-et-risque-pays/Tableau-comparatif-des-evaluations-des-pays')
df_country_risk[1].head(3)
#這裡的df是一個list of dfs 裡面包含六個dfs

,Amérique,Evaluation des risques pays,Evaluation Environnement des affaires
0,Argentine,D,B
1,Belize,D,C
2,Bolivie,C,B


In [571]:
# Il y a 6 dataframes dans ce fichier.
# Chacun est une region
for x in df_country_risk:
    print(x.columns.values)


['Afrique' 'Evaluation des risques pays'
 'Evaluation Environnement des affaires']
['Amérique' 'Evaluation des risques pays'
 'Evaluation Environnement des affaires']
['Asie' 'Evaluation des risques pays'
 'Evaluation Environnement des affaires']
['CEI' 'Evaluation des risques pays'
 'Evaluation Environnement des affaires']
['Europe' 'Evaluation des risques pays'
 'Evaluation Environnement des affaires']
['Moyen-Orient' 'Evaluation des risques pays'
 'Evaluation Environnement des affaires']


In [572]:
# On vérifie combien il y a de lignes et colonnes dans ce fichier.
for x in df_country_risk:
    print(x.shape)

(46, 3)
(27, 3)
(31, 3)
(11, 3)
(38, 3)
(14, 3)


In [573]:
# Renommer tous les colonnes comme 'Zone' pour la joiture suivante. 
#把所有的title都改成zone以利於後續作concat(union)
# scope 範圍(表示在for loop迴圈的scope內)
for x in df_country_risk:
    x.rename(columns={x.columns[0]:'Zone'},inplace=True)
    print(x.columns)

Index(['Zone', 'Evaluation des risques pays',
       'Evaluation Environnement des affaires'],
      dtype='object')
Index(['Zone', 'Evaluation des risques pays',
       'Evaluation Environnement des affaires'],
      dtype='object')
Index(['Zone', 'Evaluation des risques pays',
       'Evaluation Environnement des affaires'],
      dtype='object')
Index(['Zone', 'Evaluation des risques pays',
       'Evaluation Environnement des affaires'],
      dtype='object')
Index(['Zone', 'Evaluation des risques pays',
       'Evaluation Environnement des affaires'],
      dtype='object')
Index(['Zone', 'Evaluation des risques pays',
       'Evaluation Environnement des affaires'],
      dtype='object')


In [574]:
# On regroupe les dataframes par région dans un seul dataframe
df_country_risk = pd.concat(df_country_risk)
df_country_risk.head()

,Zone,Evaluation des risques pays,Evaluation Environnement des affaires
0,Afrique du Sud,C,A4
1,Algerie,C,C
2,Angola,C,D
3,Bénin,B,C
4,Botswana,A4,A4


L'ordre des données ci-dessus est A1, A2, A3, A4, B, C, D, E: Du plus sécurisé au plus risqué, nous devons les transformer en chiffre afin d'effectuer une analyse de corrélation ultérieure.

In [575]:
# Enlever la colonne dont on n'a pas besoin.
df_country_risk.drop(columns=['Evaluation Environnement des affaires'], axis=1, inplace = True)
# Renommer le colonne à 'CountryRisk'
df_country_risk.rename(columns={'Evaluation des risques pays':'CountryRisk'}, inplace = True)
df_country_risk.head()

,Zone,CountryRisk
0,Afrique du Sud,C
1,Algerie,C
2,Angola,C
3,Bénin,B
4,Botswana,A4


In [576]:
# Quantification des country risk 把那些指標數字化以方便後續作業
df_country_risk.replace(['A1','A2','A3','A4','B','C','D','E'],range(8), inplace= True)
df_country_risk.head(3)
#用range直接把A1到E共8個niveau直接轉換成數值了!!!!

,Zone,CountryRisk
0,Afrique du Sud,5
1,Algerie,5
2,Angola,5


我們把country risk的評分制度從0到7改回最安全是7分,最不安全是0分
中間的-1表示迴圈做到-1就會停
右邊的-1表示每次都-1
這樣才能把評分數值顛倒過來
</br>
On veut que la plus haute note corresponde aux pays les plus fiables, donc on inverse les notes.

In [577]:
df_country_risk['CountryRisk'].replace(range(7,-1,-1),range(0,8), inplace=True)

In [578]:
# Il y a 167 lignes dans 'df_country_risk'
df_country_risk.head()

,Zone,CountryRisk
0,Afrique du Sud,2
1,Algerie,2
2,Angola,2
3,Bénin,3
4,Botswana,4


In [579]:
df_country_risk.to_csv('CountryRisk.csv', index=False, encoding='utf-8')


#### Jointure
Jointure des indices de risque avec les autres données.
</br>
A cause de differences dans l'ecriture des pays ( accentuations) entre les données récupérées et celle de la FAO, on doit corriger les données un par un après la jointure.
Il y a aussi des problèmes avec des pays qui ont des noms differents.


In [580]:
# il y a 212 lignes
# On fait un outer join pour ne pas perdre de pays et vérifier ensuite
df_merged3 = df_merged2.merge(df_country_risk,how='outer',on='Zone')
df_merged3.head()

,Zone,Croissance_popu(%),population_2017,proteine_volaille(%),Production,TDI(%),TAS(%),Disponibilité alimentaire (Kcal/personne/jour),Qty_prod_hab,PIB_5ans(%),CountryRisk
0,Afghanistan,16.477889,36296.113,0.998336,28.0,50.877193,49.122807,1997.0,0.771432,-1.556597,0.0
1,Afrique du Sud,7.906280,57009.756,16.926583,1667.0,24.268178,78.706327,2987.0,29.240609,-3.981301,2.0
2,Albanie,-1.026974,2884.169,5.238494,13.0,80.851064,27.659574,3400.0,4.507364,0.723746,1.0
3,Algérie,10.713956,41389.189,2.121702,275.0,0.722022,99.277978,3345.0,6.644247,-4.103502,NaN
4,Allemagne,2.081915,82658.409,7.648698,1514.0,48.418631,87.061530,3559.0,18.316346,-0.328481,5.0


In [581]:
df_merged3.to_csv('Test.csv', index=False, encoding='utf-8')


#### Correction fusion avec indice de risque

In [582]:
mask = df_merged3["Zone"].str.contains('Algérie')
df_merged3[mask]

,Zone,Croissance_popu(%),population_2017,proteine_volaille(%),Production,TDI(%),TAS(%),Disponibilité alimentaire (Kcal/personne/jour),Qty_prod_hab,PIB_5ans(%),CountryRisk
3,Algérie,10.713956,41389.189,2.121702,275.0,0.722022,99.277978,3345.0,6.644247,-4.103502,NaN


In [583]:
# Remplir le data de CountryRisk pour 'Algérie'
df_merged3.loc[3,'CountryRisk'] = 2.0
# Supprimer le duplicated'Algérie'
df_merged3.drop(axis=0,index=172,inplace=True)
# Remplir le data de CountryRisk pour Cabo Verde
df_merged3.loc[26,'CountryRisk'] = 2.0
# Supprimer le duplicated'Cabo Verde'
df_merged3.drop(axis=0,index=174,inplace=True)
# Enlever duplicated 'Congo'
df_merged3.drop(axis=0,index=175,inplace=True)
# Remplir le data de CountryRisk pour Égypte
df_merged3.loc[166,'CountryRisk'] = 2.0
# Supprimer le duplicated pour Égypte
df_merged3.drop(axis=0,index=176,inplace=True)
# Remplir le data de CountryRisk pour Éthiopie
df_merged3.loc[170,'CountryRisk'] = 2.0
# Supprimer le duplicated pour Éthiopie
df_merged3.drop(axis=0,index=178,inplace=True)
# Remplir le data de CountryRisk pour Libéria
df_merged3.loc[85,'CountryRisk'] = 1.0
# Supprimer le duplicated pour Libéria
df_merged3.drop(axis=0,index=179,inplace=True)
# Remplir le data de CountryRisk pour Nigéria
df_merged3.loc[106,'CountryRisk'] = 1.0
# Supprimer le duplicated pour Nigéria
df_merged3.drop(axis=0,index=181,inplace=True)
# Remplir le data de CountryRisk pour République-Unie de Tanzanie
df_merged3.loc[132,'CountryRisk'] = 2.0
# Supprimer le duplicated pour République-Unie de Tanzanie
df_merged3.drop(axis=0,index=182,inplace=True)
# Remplir le data de CountryRisk pour Bolivie (État plurinational de)
df_merged3.loc[18,'CountryRisk'] = 2.0
# Supprimer le duplicated pour Bolivie (État plurinational de)
df_merged3.drop(axis=0,index=183,inplace=True)
# Remplir le data de CountryRisk pour États-Unis d'Amérique
df_merged3.loc[169,'CountryRisk'] = 6.0
# Supprimer le duplicated pour États-Unis d'Amérique
df_merged3.drop(axis=0,index=184,inplace=True)
# Remplir le data de CountryRisk pour El Salvador
df_merged3.loc[45,'CountryRisk'] = 1.0
# Supprimer le duplicated pour El Salvador
df_merged3.drop(axis=0,index=185,inplace=True)
# Remplir le data de CountryRisk pour Venezuela (République bolivarienne du)
df_merged3.loc[161,'CountryRisk'] = 0.0
# Supprimer le duplicated pour Venezuela (République bolivarienne du)
df_merged3.drop(axis=0,index=186,inplace=True)
# Remplir le data de CountryRisk pour République de Corée
df_merged3.loc[127,'CountryRisk'] = 6.0
# Supprimer le duplicated pour République de Corée
df_merged3.drop(axis=0,index=189,inplace=True)
# Supprimer Corée du Nord 考量到這個國家和法國沒有貿易往來
df_merged3.drop(axis=0,index=131,inplace=True)
df_merged3.drop(axis=0,index=188,inplace=True)
# Remplir le data de CountryRisk pour Chine - RAS de Hong-Kong
df_merged3.loc[31,'CountryRisk'] = 5.0
# Supprimer le duplicated pour Chine - RAS de Hong-Kong
df_merged3.drop(axis=0,index=190,inplace=True)
# Remplir le data de CountryRisk pour Indonésie	
df_merged3.loc[67,'CountryRisk'] = 4.0
# Supprimer le duplicated pour Indonésie	
df_merged3.drop(axis=0,index=191,inplace=True)
# Remplir le data de CountryRisk pour République démocratique populaire lao
df_merged3.loc[130,'CountryRisk'] = 1.0
# Supprimer le duplicated pour République démocratique populaire lao
df_merged3.drop(axis=0,index=192,inplace=True)
# Remplir le data de CountryRisk pour Eswatini	
df_merged3.loc[48,'CountryRisk'] = 1.0
# Supprimer le duplicated pour Eswatini(Swaziland)	
df_merged3.drop(axis=0,index=195,inplace=True)
# Remplir le data de CountryRisk pour Taiwan
df_merged3.loc[33,'CountryRisk'] = 6.0
# Supprimer le duplicated pour Taiwan
df_merged3.drop(axis=0,index=196,inplace=True)
# Remplir le data de CountryRisk pour Timor-Leste
df_merged3.loc[152,'CountryRisk'] = 1.0
# Supprimer le duplicated pour Timor-Leste
df_merged3.drop(axis=0,index=197,inplace=True)
# Remplir le data de CountryRisk pour Viet Nam	
df_merged3.loc[162,'CountryRisk'] = 3.0
# Supprimer le duplicated pour Viet Nam	
df_merged3.drop(axis=0,index=198,inplace=True)
# Supprimer Syrie car il n'existe pas dans df_merged3
df_merged3.drop(axis=0,index=211,inplace=True)
# Remplir le data de CountryRisk pour Azerbaïdjan	
df_merged3.loc[12,'CountryRisk'] = 3.0
# Supprimer le duplicated pour Azerbaïdjan (Azerbaijan)	
df_merged3.drop(axis=0,index=199,inplace=True)
# Remplir le data de CountryRisk pour République de Moldova
df_merged3.loc[128,'CountryRisk'] = 2.0
# Supprimer le duplicated pour République de Moldova
df_merged3.drop(axis=0,index=200,inplace=True)
# Remplir le data de CountryRisk pour Fédération de Russie	
df_merged3.loc[52,'CountryRisk'] = 1.0
# Supprimer le duplicated pour Fédération de Russie	
df_merged3.drop(axis=0,index=201,inplace=True)
# Remplir le data de CountryRisk pour Bélarus (Biélorussie)	
df_merged3.loc[24,'CountryRisk'] = 1.0
# Supprimer le duplicated pour Bélarus (Biélorussie)		
df_merged3.drop(axis=0,index=202,inplace=True)
# Remplir le data de CountryRisk pour Monténégro
df_merged3.loc[100,'CountryRisk'] =2.0
# Supprimer le duplicated pour Monténégro
df_merged3.drop(axis=0,index=203,inplace=True)
# Remplir le data de CountryRisk pour Royaume-Uni de Grande-Bretagne et d'Irlande du(英國)
df_merged3.loc[124,'CountryRisk'] =4.0
# Supprimer le duplicated pour Royaume-Uni de Grande-Bretagne et d'Irlande du(英國)
df_merged3.drop(axis=0,index=204,inplace=True)
# Remplir le data de CountryRisk pour Tchéquie	
df_merged3.loc[150,'CountryRisk'] = 4.0
# Supprimer le duplicated pour Tchéquie	
df_merged3.drop(axis=0,index=205,inplace=True)
# Supprimer Bahrein car il n'existe pas dans df_merged3
df_merged3.drop(axis=0,index=206,inplace=True)
# Remplir le data de CountryRisk pour Iran (République islamique d')
df_merged3.loc[68,'CountryRisk'] = 0.0
# Supprimer le duplicated pour Iran (République islamique d')
df_merged3.drop(axis=0,index=208,inplace=True)
# Remplir le data de CountryRisk pour Iraq	
df_merged3.loc[69,'CountryRisk'] = 0.0
# Supprimer le duplicated pour Iraq	
df_merged3.drop(axis=0,index=207,inplace=True)
# Supprimer Palestine (Territoires)	 car il n'existe pas dans df_merged3
df_merged3.drop(axis=0,index=209,inplace=True)
# Remplir le data de CountryRisk pour 'Chine, continentale'
df_merged3.loc[34,'CountryRisk'] = 3.0
# Supprimer le duplicated'Chine, continentale'
df_merged3.drop(axis=0,index=187,inplace=True)
# Supprimer Burundi	car il n'existe pas dans df_merged3
df_merged3.drop(axis=0,index=173,inplace=True)
# Supprimer Érythrée car il n'existe pas dans df_merged3
df_merged3.drop(axis=0,index=177,inplace=True)
# Supprimer Libye car il n'existe pas dans df_merged3
df_merged3.drop(axis=0,index=180,inplace=True)
# Supprimer Papouasie-Nouvelle-Guinée car il n'existe pas dans df_merged3
df_merged3.drop(axis=0,index=193,inplace=True)
# Supprimer Singapour car il n'existe pas dans df_merged3
df_merged3.drop(axis=0,index=194,inplace=True)	
# Supprimer Qatar car il n'existe pas dans df_merged3
df_merged3.drop(axis=0,index=210,inplace=True)

In [584]:
# Il y a 171 lignes.
df_merged3.shape

(171, 11)

#### Traitement des valeurs nulles de l'indice risque

On a 15 pays qui n'ont pas d'indice risque

In [585]:
df_merged3.isnull().sum()

Zone                                               0
Croissance_popu(%)                                 0
population_2017                                    0
proteine_volaille(%)                               0
Production                                         0
TDI(%)                                             0
TAS(%)                                             0
Disponibilité alimentaire (Kcal/personne/jour)     0
Qty_prod_hab                                       0
PIB_5ans(%)                                        0
CountryRisk                                       15
dtype: int64

In [586]:
# Fillna la moyenne dans les valeurs nulles de la colonne 'CountryRisk'
df_merged3["CountryRisk"] = df_merged3["CountryRisk"].fillna(df_merged3["CountryRisk"].mean())

In [587]:
df_merged3.isnull().sum()

Zone                                              0
Croissance_popu(%)                                0
population_2017                                   0
proteine_volaille(%)                              0
Production                                        0
TDI(%)                                            0
TAS(%)                                            0
Disponibilité alimentaire (Kcal/personne/jour)    0
Qty_prod_hab                                      0
PIB_5ans(%)                                       0
CountryRisk                                       0
dtype: int64

In [588]:
df_merged3.head()

,Zone,Croissance_popu(%),population_2017,proteine_volaille(%),Production,TDI(%),TAS(%),Disponibilité alimentaire (Kcal/personne/jour),Qty_prod_hab,PIB_5ans(%),CountryRisk
0,Afghanistan,16.477889,36296.113,0.998336,28.0,50.877193,49.122807,1997.0,0.771432,-1.556597,0.0
1,Afrique du Sud,7.906280,57009.756,16.926583,1667.0,24.268178,78.706327,2987.0,29.240609,-3.981301,2.0
2,Albanie,-1.026974,2884.169,5.238494,13.0,80.851064,27.659574,3400.0,4.507364,0.723746,1.0
3,Algérie,10.713956,41389.189,2.121702,275.0,0.722022,99.277978,3345.0,6.644247,-4.103502,2.0
4,Allemagne,2.081915,82658.409,7.648698,1514.0,48.418631,87.061530,3559.0,18.316346,-0.328481,5.0


### EEE : L'Espace économique européen 
(EEE, ou EEA en anglais)
On va récupérer la liste des pays de l'EEE sur wikipédia.
</br>
*Source :Wikipedia*

In [589]:
# Web Scraping
# pandas方便抓取所有<table>包含的資料,因為其呈現dataframe的模式
import pandas as pd
df_list = pd.read_html('https://fr.wikipedia.org/wiki/Espace_%C3%A9conomique_europ%C3%A9en')
df_EEE= df_list[2]
df_EEE.head()

,État,"Signature[Note 1],[6],[7]","Ratification[Note 1],[6]",Entrée en vigueur[6],Notes
0,Allemagne,2 mai 1992,23 juin 1993,1er janvier 1994,UE
1,Autriche,2 mai 1992,15 octobre 1992,1er janvier 1994,UE (depuis 1er janvier 1995)A adhéré à l'EEE e...
2,Belgique,2 mai 1992,9 novembre 1993,1er janvier 1994,UE
3,Bulgarie[8],25 juillet 2007,29 février 2008,9 novembre 2011,UE
4,Chypre[9],14 octobre 2003,30 avril 2004,6 décembre 2005,UE


In [590]:
# Enlever les colonnes dont on n'a pas besoin
df_EEE.drop(columns=['Signature[Note 1],[6],[7]','Ratification[Note 1],[6]','Entrée en vigueur[6]','Notes'], axis=1, inplace = True)

In [591]:
df_EEE.rename(columns={'État':'Zone'},inplace= True)
df_EEE.head()

,Zone
0,Allemagne
1,Autriche
2,Belgique
3,Bulgarie[8]
4,Chypre[9]


In [592]:
# On supprime les patterns "[8]" dans la colonne de 'Zone' avec une regex (regular expression)
# On remplace les morceaux de phrases qui ont "[" + des chiffres + "]" par ""
df_EEE.loc[:,'Zone'] =df_EEE['Zone'].str.replace(r'\[[0-9]*\]','', regex=True)


In [593]:
#Supprimez les deux dernières lignes redondantes
#刪除掉最下面兩行多餘的rows
df_EEE.drop(labels=[32], axis ='index',inplace= True)

In [594]:
# Ajout de colonne a la fin "EEA" avec la valeur 1 ( == fait partie de l'EEE)
df_EEE.insert(df_EEE.shape[1],'EEA','1')

In [595]:
# il y a 31 pays dans df_EEE
df_EEE.head(3)

,Zone,EEA
0,Allemagne,1
1,Autriche,1
2,Belgique,1


In [596]:
df_EEE.to_csv('EEA_members.csv', index=False, encoding='utf-8')

#### Jointure de l'EEA 
On merge l'EEA dans le dataframe

In [597]:
df_merged4 = df_merged3.merge(df_EEE,how='outer',on='Zone')
df_merged4

,Zone,Croissance_popu(%),population_2017,proteine_volaille(%),Production,TDI(%),TAS(%),Disponibilité alimentaire (Kcal/personne/jour),Qty_prod_hab,PIB_5ans(%),CountryRisk,EEA
0,Afghanistan,16.477889,36296.113,0.998336,28.0,50.877193,49.122807,1997.0,0.771432,-1.556597,0.000000,NaN
1,Afrique du Sud,7.906280,57009.756,16.926583,1667.0,24.268178,78.706327,2987.0,29.240609,-3.981301,2.000000,NaN
2,Albanie,-1.026974,2884.169,5.238494,13.0,80.851064,27.659574,3400.0,4.507364,0.723746,1.000000,NaN
3,Algérie,10.713956,41389.189,2.121702,275.0,0.722022,99.277978,3345.0,6.644247,-4.103502,2.000000,NaN
4,Allemagne,2.081915,82658.409,7.648698,1514.0,48.418631,87.061530,3559.0,18.316346,-0.328481,5.000000,1
...,...,...,...,...,...,...,...,...,...,...,...,...
168,États-Unis d'Amérique,3.515710,325084.756,17.542470,21914.0,0.673382,119.971532,3762.0,67.410113,3.041601,6.000000,NaN
169,Éthiopie,14.745372,106399.924,0.060938,14.0,7.142857,100.000000,2304.0,0.131579,13.872441,2.000000,NaN
170,Îles Salomon,14.382543,636.039,2.815063,0.0,200.000000,0.000000,2410.0,0.000000,2.903426,2.673077,NaN
171,Liechtenstein,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [598]:
mask = df_merged4["Zone"].str.contains('Liechtenstein')
df_merged4[mask]

,Zone,Croissance_popu(%),population_2017,proteine_volaille(%),Production,TDI(%),TAS(%),Disponibilité alimentaire (Kcal/personne/jour),Qty_prod_hab,PIB_5ans(%),CountryRisk,EEA
171,Liechtenstein,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [599]:
# Supprimer Liechtenstein car on a pas de données
df_merged4.drop(axis=0,index=171,inplace=True)

In [600]:
mask = df_merged4["Zone"].str.contains('Royaume-Uni')
df_merged4[mask]

,Zone,Croissance_popu(%),population_2017,proteine_volaille(%),Production,TDI(%),TAS(%),Disponibilité alimentaire (Kcal/personne/jour),Qty_prod_hab,PIB_5ans(%),CountryRisk,EEA
124,Royaume-Uni de Grande-Bretagne et d'Irlande du...,3.412845,66727.461,13.256956,1814.0,34.870188,81.199642,3428.0,27.185209,-0.379473,4.0,NaN
172,Royaume-Uni,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [601]:
# Remplir le data de EEA pour Royaume-Uni de Grande-Bretagne et d'Irlande du...
df_merged4.loc[124,'EEA'] = 1
# Supprimer le duplicated Royaume-Uni de Grande-Bretagne et d'Irlande du...
df_merged4.drop(axis=0,index=172,inplace=True)


- Pour les pays de l'EEE, on a la valeur 1
- Pour ceux hors de l'EEE, on remplit avec des 0

In [602]:
# On remplit 0 dans la colonne 'EEA' pour les pays qui ne sont pas dans EEA.
df_merged4["EEA"] = df_merged4["EEA"].fillna(0)
df_merged4.isnull().sum()

Zone                                              0
Croissance_popu(%)                                0
population_2017                                   0
proteine_volaille(%)                              0
Production                                        0
TDI(%)                                            0
TAS(%)                                            0
Disponibilité alimentaire (Kcal/personne/jour)    0
Qty_prod_hab                                      0
PIB_5ans(%)                                       0
CountryRisk                                       0
EEA                                               0
dtype: int64

In [603]:
df_merged4.head()

,Zone,Croissance_popu(%),population_2017,proteine_volaille(%),Production,TDI(%),TAS(%),Disponibilité alimentaire (Kcal/personne/jour),Qty_prod_hab,PIB_5ans(%),CountryRisk,EEA
0,Afghanistan,16.477889,36296.113,0.998336,28.0,50.877193,49.122807,1997.0,0.771432,-1.556597,0.0,0
1,Afrique du Sud,7.906280,57009.756,16.926583,1667.0,24.268178,78.706327,2987.0,29.240609,-3.981301,2.0,0
2,Albanie,-1.026974,2884.169,5.238494,13.0,80.851064,27.659574,3400.0,4.507364,0.723746,1.0,0
3,Algérie,10.713956,41389.189,2.121702,275.0,0.722022,99.277978,3345.0,6.644247,-4.103502,2.0,0
4,Allemagne,2.081915,82658.409,7.648698,1514.0,48.418631,87.061530,3559.0,18.316346,-0.328481,5.0,1


In [604]:
df_merged4.to_csv('dfmerged4.csv', index=False, encoding='utf-8')

### La distance à la France

Pour la distance, on télécharge le fichier "distances.csv" sur le répertoire github (voir source)
</br>
Les pays sont indiqués avec leur code ISO3. 

source : https://github.com/rahulbot/distances-between-countries

In [605]:
# Importer le fichier distance a la France
df_distance= pd.read_csv('distances.csv')
df_distance.head()

,,ABW,AFG,AGO,AIA,ALB,AND,ARE,ARG,ARM,...,VCT,VEN,VNM,VUT,WLF,WSM,YEM,ZAF,ZMB,ZWE
0,ABW,0.000000,13252.562557,9500.724683,973.535120,9085.425408,7565.414510,12767.098434,5360.114081,11105.217199,...,953.217628,407.254233,17394.630809,13764.066135,12057.320301,11596.227190,12153.647425,10635.960050,11252.792271,11587.659716
1,AFG,13252.562557,0.000000,7592.695548,12295.402341,4344.887482,5809.607175,1685.264881,15269.257992,2264.595710,...,12582.312572,13213.101745,4615.406201,11930.663622,13090.899638,13527.156291,3279.105626,9242.212938,7017.607917,7060.695339
2,AGO,9500.724683,7592.695548,0.000000,8894.790019,5594.945971,5808.941318,5917.159476,7777.930733,6309.127205,...,8581.313382,9127.152398,10567.405666,16001.483832,17350.882991,17438.154618,4339.161563,2828.661088,1791.381489,2169.266415
3,AIA,973.535120,12295.402341,8894.790019,0.000000,8111.936087,6591.882815,11797.582285,5867.656461,10134.035979,...,593.198987,949.165376,16601.574937,14610.029750,12882.443919,12434.627974,11234.495290,10369.917631,10679.299892,11038.903154
4,ALB,9085.425408,4344.887482,5594.945971,8111.936087,0.000000,1520.615798,3718.578045,11591.279736,2082.227281,...,8313.202282,8974.865549,8960.200423,16037.392002,16534.505765,16759.477394,3714.765270,8333.606713,6341.330364,6650.773482


In [606]:
df_distance.columns.values

array([' ', 'ABW', 'AFG', 'AGO', 'AIA', 'ALB', 'AND', 'ARE', 'ARG', 'ARM',
       'ASM', 'ATA', 'ATF', 'ATG', 'AUS', 'AUT', 'AZE', 'BDI', 'BEL',
       'BEN', 'BES', 'BFA', 'BGD', 'BGR', 'BHR', 'BHS', 'BIH', 'BLM',
       'BLR', 'BLZ', 'BMU', 'BOL', 'BRA', 'BRB', 'BTN', 'BWA', 'CAF',
       'CAN', 'CCK', 'CHE', 'CHL', 'CHN', 'CIV', 'CMR', 'COD', 'COG',
       'COK', 'COL', 'COM', 'CPV', 'CRI', 'CUB', 'CUW', 'CXR', 'CYM',
       'CYP', 'CZE', 'DEU', 'DJI', 'DMA', 'DNK', 'DOM', 'DZA', 'ECU',
       'EGY', 'ERI', 'ESH', 'ESP', 'EST', 'ETH', 'FIN', 'FJI', 'FRA',
       'FRO', 'FSM', 'GAB', 'GBR', 'GEO', 'GGY', 'GHA', 'GIB', 'GIN',
       'GLP', 'GMB', 'GNB', 'GNQ', 'GRC', 'GRD', 'GRL', 'GTM', 'GUF',
       'GUM', 'GUY', 'HKG', 'HND', 'HRV', 'HTI', 'HUN', 'IDN', 'IMN',
       'IND', 'IOT', 'IRL', 'IRN', 'IRQ', 'ISL', 'ISR', 'ITA', 'JAM',
       'JEY', 'JOR', 'JPN', 'KAZ', 'KEN', 'KGZ', 'KHM', 'KIR', 'KNA',
       'KWT', 'LBN', 'LBR', 'LBY', 'LCA', 'LIE', 'LKA', 'LSO', 'LTU',
       'LUX', '

In [607]:
df_distance.index.names

FrozenList([None])

In [608]:
df_distance1 = df_distance.rename(columns= {df_distance.columns.values[0]:'iso3'})
df_distance1.head()

,iso3,ABW,AFG,AGO,AIA,ALB,AND,ARE,ARG,ARM,...,VCT,VEN,VNM,VUT,WLF,WSM,YEM,ZAF,ZMB,ZWE
0,ABW,0.000000,13252.562557,9500.724683,973.535120,9085.425408,7565.414510,12767.098434,5360.114081,11105.217199,...,953.217628,407.254233,17394.630809,13764.066135,12057.320301,11596.227190,12153.647425,10635.960050,11252.792271,11587.659716
1,AFG,13252.562557,0.000000,7592.695548,12295.402341,4344.887482,5809.607175,1685.264881,15269.257992,2264.595710,...,12582.312572,13213.101745,4615.406201,11930.663622,13090.899638,13527.156291,3279.105626,9242.212938,7017.607917,7060.695339
2,AGO,9500.724683,7592.695548,0.000000,8894.790019,5594.945971,5808.941318,5917.159476,7777.930733,6309.127205,...,8581.313382,9127.152398,10567.405666,16001.483832,17350.882991,17438.154618,4339.161563,2828.661088,1791.381489,2169.266415
3,AIA,973.535120,12295.402341,8894.790019,0.000000,8111.936087,6591.882815,11797.582285,5867.656461,10134.035979,...,593.198987,949.165376,16601.574937,14610.029750,12882.443919,12434.627974,11234.495290,10369.917631,10679.299892,11038.903154
4,ALB,9085.425408,4344.887482,5594.945971,8111.936087,0.000000,1520.615798,3718.578045,11591.279736,2082.227281,...,8313.202282,8974.865549,8960.200423,16037.392002,16534.505765,16759.477394,3714.765270,8333.606713,6341.330364,6650.773482


In [609]:
# On ne veut que la distance entre le pays et la France
distance_france = df_distance1.loc[:,['iso3','FRA']]
distance_france

,iso3,FRA
0,ABW,7677.486897
1,AFG,5594.223377
2,AGO,6478.953877
3,AIA,6710.883702
4,ALB,1603.944535
...,...,...
230,WSM,16083.468551
231,YEM,5311.607947
232,ZAF,9306.638530
233,ZMB,7565.182929


On a besoin de faire le lien entre le code ISO3 et le nom des pays. Pour cela on récupère les données sur le site https://www.atlas-monde.net/

In [610]:
# On importe la fichier pour les pays corespondent aux codes de leur Iso3
df_ISO3 = pd.read_html('https://www.atlas-monde.net/codes-iso/',header=None)
df_ISO3= df_ISO3[0]
df_ISO3.head()


,0,1,2,3
0,Pays,ISO 3166-1 alpha-2,ISO 3166-1 alpha-3,ISO 3166-1 numérique
1,Afghanistan,AF,AFG,4
2,Îles Åland,AX,ALA,248
3,Albanie,AL,ALB,8
4,Algérie,DZ,DZA,12


In [611]:
# On ajoute le nom pour chaque colonne 放入column頭名 
df_ISO3.columns=['Zone','ISO 3166-1 alpha-2','iso3','ISO 3166-1 numérique']
# On supprime la premiere ligne
df_ISO3.drop(axis=0,index=0,inplace=True)

In [612]:
# On conserve les colonnes dont on a besoin
df_ISO3 = df_ISO3.loc[:,['iso3','Zone']]
df_ISO3.head()

,iso3,Zone
1,AFG,Afghanistan
2,ALA,Îles Åland
3,ALB,Albanie
4,DZA,Algérie
5,ASM,Samoa américaines


In [613]:
df_distance2 = df_ISO3.merge(distance_france,how= 'inner', on='iso3')
df_distance2.head()

,iso3,Zone,FRA
0,AFG,Afghanistan,5594.223377
1,ALB,Albanie,1603.944535
2,DZA,Algérie,1345.460397
3,ASM,Samoa américaines,16117.698909
4,AND,Andorre,708.213661


In [614]:
# On change la position de colonnes, 'Zone' devients la premiere
cols = df_distance2.columns.tolist()
cols.insert(0,cols.pop(cols.index('Zone')))
df_distance2 = df_distance2[cols]

In [615]:
df_distance2.head()

,Zone,iso3,FRA
0,Afghanistan,AFG,5594.223377
1,Albanie,ALB,1603.944535
2,Algérie,DZA,1345.460397
3,Samoa américaines,ASM,16117.698909
4,Andorre,AND,708.213661


In [435]:
df_distance2.to_csv('Distance_france.csv', index=False, encoding='utf-8')

#### Jointure de la distance
df_merged5 + df_distance2

In [616]:
# Il y a 171 pays dans df_merged5
df_merged5 = df_merged4.merge(df_distance2,how='left',on='Zone')
df_merged5.head()

,Zone,Croissance_popu(%),population_2017,proteine_volaille(%),Production,TDI(%),TAS(%),Disponibilité alimentaire (Kcal/personne/jour),Qty_prod_hab,PIB_5ans(%),CountryRisk,EEA,iso3,FRA
0,Afghanistan,16.477889,36296.113,0.998336,28.0,50.877193,49.122807,1997.0,0.771432,-1.556597,0.0,0,AFG,5594.223377
1,Afrique du Sud,7.906280,57009.756,16.926583,1667.0,24.268178,78.706327,2987.0,29.240609,-3.981301,2.0,0,ZAF,9306.638530
2,Albanie,-1.026974,2884.169,5.238494,13.0,80.851064,27.659574,3400.0,4.507364,0.723746,1.0,0,ALB,1603.944535
3,Algérie,10.713956,41389.189,2.121702,275.0,0.722022,99.277978,3345.0,6.644247,-4.103502,2.0,0,DZA,1345.460397
4,Allemagne,2.081915,82658.409,7.648698,1514.0,48.418631,87.061530,3559.0,18.316346,-0.328481,5.0,1,DEU,880.634838


#### On supprime les pays de moins de 5 millions d'habitants
Pour l'analyse, on ne regarde pas les pays ayant un trop faible nombre de population.

In [617]:
df_merged5 = df_merged5[~(df_merged5['population_2017'] <= 500)]
df_merged5.head()

,Zone,Croissance_popu(%),population_2017,proteine_volaille(%),Production,TDI(%),TAS(%),Disponibilité alimentaire (Kcal/personne/jour),Qty_prod_hab,PIB_5ans(%),CountryRisk,EEA,iso3,FRA
0,Afghanistan,16.477889,36296.113,0.998336,28.0,50.877193,49.122807,1997.0,0.771432,-1.556597,0.0,0,AFG,5594.223377
1,Afrique du Sud,7.906280,57009.756,16.926583,1667.0,24.268178,78.706327,2987.0,29.240609,-3.981301,2.0,0,ZAF,9306.638530
2,Albanie,-1.026974,2884.169,5.238494,13.0,80.851064,27.659574,3400.0,4.507364,0.723746,1.0,0,ALB,1603.944535
3,Algérie,10.713956,41389.189,2.121702,275.0,0.722022,99.277978,3345.0,6.644247,-4.103502,2.0,0,DZA,1345.460397
4,Allemagne,2.081915,82658.409,7.648698,1514.0,48.418631,87.061530,3559.0,18.316346,-0.328481,5.0,1,DEU,880.634838


#### Remplissage de données manquantes
 On remplit les données correspondantes manuelement pour les pays avec des données manquantes.
 On a ce problème pour la distance et l'ISO3

In [618]:
# Arabie saoudite
df_merged5.loc[7,'iso3'] = 'SAU'
df_merged5.loc[7,'FRA'] = 4686.618146

In [619]:
# Bolivie (État plurinational de)	
df_merged5.loc[18,'iso3'] = 'BOL'
df_merged5.loc[18,'FRA'] = 9804.137957	

In [620]:
# Bélarus
df_merged5.loc[24,'iso3'] = 'BLR'
df_merged5.loc[24,'FRA'] = 1832.066997

In [621]:
# Cabo Verde	
df_merged5.loc[26,'iso3'] = 'CPV'
df_merged5.loc[26,'FRA'] = 4441.01822252098

In [622]:
# Chine - RAS de Hong-Kong	
df_merged5.loc[31,'iso3'] = 'HKG'
df_merged5.loc[31,'FRA'] = 9645.8558478014

In [623]:
# Chine - RAS de Macao
df_merged5.loc[32,'iso3'] = 'MAC'
df_merged5.loc[32,'FRA'] = 9614.59290609614

In [624]:
# Chine, Taiwan Province de
df_merged5.loc[33,'iso3'] = 'TWN'
df_merged5.loc[33,'FRA'] = 9846.95340013131

In [625]:
# Chine, continentale	
df_merged5.loc[34,'iso3'] = 'CHN'
df_merged5.loc[34,'FRA'] = 9283.66837451508

In [629]:
# Congo	
df_merged5.loc[37,'iso3'] = 'COD'
df_merged5.loc[37,'FRA'] = 5786.76115325604

In [627]:
# Côte d'Ivoire	
df_merged5.loc[41,'iso3'] = 'CIV'
df_merged5.loc[41,'FRA'] = 4644.913554	

In [628]:
# Fédération de Russie	
df_merged5.loc[52,'iso3'] = 'RUS'
df_merged5.loc[52,'FRA'] = 2493.922256		

In [630]:
# États-Unis d'Amérique
df_merged5.loc[168,'iso3'] = 'USA'
df_merged5.loc[168,'FRA'] = 5852.736948	

In [631]:
# République démocratique populaire lao
df_merged5.loc[130,'iso3'] = 'LAO'
df_merged5.loc[130,'FRA'] = 9236.35

In [632]:
# République de Corée
df_merged5.loc[127,'iso3'] = 'KOR'
df_merged5.loc[127,'FRA'] = 8966.56

In [633]:
# Viet Nam	
df_merged5.loc[161,'iso3'] = 'VNM'
df_merged5.loc[161,'FRA'] = 10132.8144597859

In [634]:
# Tchéquie	
df_merged5.loc[149,'iso3'] = 'CZE'
df_merged5.loc[149,'FRA'] = 884.671277

In [635]:
# Émirats arabes unis	
df_merged5.loc[166,'iso3'] = 'ARE'
df_merged5.loc[166,'FRA'] = 5251.803242

In [636]:
# Venezuela (République bolivarienne du)	
df_merged5.loc[160,'iso3'] = 'VEN'
df_merged5.loc[160,'FRA'] = 7619.687246

In [637]:
# République-Unie de Tanzanie	
df_merged5.loc[131,'iso3'] = 'TZA'
df_merged5.loc[131,'FRA'] = 7138.574822

In [638]:
# El Salvador	
df_merged5.loc[45,'iso3'] = 'SLV'
df_merged5.loc[45,'FRA'] = 8986.492433

In [639]:
# Guyana
df_merged5.loc[61,'iso3'] = 'GUY'
df_merged5.loc[61,'FRA'] = 7308.347596

In [640]:
# Iran (République islamique d')
df_merged5.loc[68,'iso3'] = 'IRN'
df_merged5.loc[68,'FRA'] = 4219.837255

In [641]:
# Iraq	
df_merged5.loc[69,'iso3'] = 'IRQ'
df_merged5.loc[69,'FRA'] = 3866.519023

In [642]:
# Macédoine du Nord	
df_merged5.loc[88,'iso3'] = 'MKD'
df_merged5.loc[88,'FRA'] = 1670.248031

In [643]:
# Nigéria
df_merged5.loc[106,'iso3'] = 'NGA'
df_merged5.loc[106,'FRA'] = 4700.791412

In [644]:
# Royaume-Uni de Grande-Bretagne et d'Irlande du...
df_merged5.loc[124,'iso3'] = 'GBR'
df_merged5.loc[124,'FRA'] = 344.136719

In [645]:
# République de Moldova
df_merged5.loc[128,'iso3'] = 'MDA'
df_merged5.loc[128,'FRA'] = 1981.378059

In [646]:
df_merged5[df_merged5.isna().any(axis=1)]

,Zone,Croissance_popu(%),population_2017,proteine_volaille(%),Production,TDI(%),TAS(%),Disponibilité alimentaire (Kcal/personne/jour),Qty_prod_hab,PIB_5ans(%),CountryRisk,EEA,iso3,FRA


In [647]:
# On change le nom de colonne 'FRA' à 'Distance'.
df_merged5 = df_merged5.rename(columns= {'FRA':'Distance'})
df_merged5.head()

,Zone,Croissance_popu(%),population_2017,proteine_volaille(%),Production,TDI(%),TAS(%),Disponibilité alimentaire (Kcal/personne/jour),Qty_prod_hab,PIB_5ans(%),CountryRisk,EEA,iso3,Distance
0,Afghanistan,16.477889,36296.113,0.998336,28.0,50.877193,49.122807,1997.0,0.771432,-1.556597,0.0,0,AFG,5594.223377
1,Afrique du Sud,7.906280,57009.756,16.926583,1667.0,24.268178,78.706327,2987.0,29.240609,-3.981301,2.0,0,ZAF,9306.638530
2,Albanie,-1.026974,2884.169,5.238494,13.0,80.851064,27.659574,3400.0,4.507364,0.723746,1.0,0,ALB,1603.944535
3,Algérie,10.713956,41389.189,2.121702,275.0,0.722022,99.277978,3345.0,6.644247,-4.103502,2.0,0,DZA,1345.460397
4,Allemagne,2.081915,82658.409,7.648698,1514.0,48.418631,87.061530,3559.0,18.316346,-0.328481,5.0,1,DEU,880.634838


In [648]:
# On change le nom de Taiwan
df_merged5.loc[33,'Zone'] = 'Taiwan'

# On change le nom de Chine
df_merged5.loc[34,'Zone'] = 'Chine'

## Creation du fichier finale

On a terminé le nettoyage et la preparation des données.
Il se compose des colonnes suivantes:


In [649]:
df_merged5.isnull().sum()

Zone                                              0
Croissance_popu(%)                                0
population_2017                                   0
proteine_volaille(%)                              0
Production                                        0
TDI(%)                                            0
TAS(%)                                            0
Disponibilité alimentaire (Kcal/personne/jour)    0
Qty_prod_hab                                      0
PIB_5ans(%)                                       0
CountryRisk                                       0
EEA                                               0
iso3                                              0
Distance                                          0
dtype: int64

In [650]:
df_merged5

,Zone,Croissance_popu(%),population_2017,proteine_volaille(%),Production,TDI(%),TAS(%),Disponibilité alimentaire (Kcal/personne/jour),Qty_prod_hab,PIB_5ans(%),CountryRisk,EEA,iso3,Distance
0,Afghanistan,16.477889,36296.113,0.998336,28.0,50.877193,49.122807,1997.0,0.771432,-1.556597,0.000000,0,AFG,5594.223377
1,Afrique du Sud,7.906280,57009.756,16.926583,1667.0,24.268178,78.706327,2987.0,29.240609,-3.981301,2.000000,0,ZAF,9306.638530
2,Albanie,-1.026974,2884.169,5.238494,13.0,80.851064,27.659574,3400.0,4.507364,0.723746,1.000000,0,ALB,1603.944535
3,Algérie,10.713956,41389.189,2.121702,275.0,0.722022,99.277978,3345.0,6.644247,-4.103502,2.000000,0,DZA,1345.460397
4,Allemagne,2.081915,82658.409,7.648698,1514.0,48.418631,87.061530,3559.0,18.316346,-0.328481,5.000000,1,DEU,880.634838
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166,Émirats arabes unis,3.780598,9487.203,13.798247,48.0,105.097087,11.650485,3370.0,5.059447,0.798578,5.000000,0,ARE,5251.803242
167,Équateur,8.473901,16785.361,9.350768,340.0,0.000000,99.706745,2585.0,20.255745,3.110131,2.000000,0,ECU,9626.173816
168,États-Unis d'Amérique,3.515710,325084.756,17.542470,21914.0,0.673382,119.971532,3762.0,67.410113,3.041601,6.000000,0,USA,5852.736948
169,Éthiopie,14.745372,106399.924,0.060938,14.0,7.142857,100.000000,2304.0,0.131579,13.872441,2.000000,0,ETH,5566.295819


In [651]:
df_merged5.to_csv('finale.csv', index=False, encoding='utf-8')